In [1]:
import sqlite3
import pandas as pd

In [2]:
path="yt_project.db"
timeout = 30  # Увеличение таймаута до 30 секунд

### Триггер на мониторинг доходности по проекту

Добавить колонку статуса

In [3]:
with sqlite3.connect(path, timeout=timeout) as conn:
    cursor = conn.cursor()
    cursor.execute("ALTER TABLE Projects ADD COLUMN Status TEXT;")

Ручной пересчет статуса проекта

In [4]:
def recalculate_project_status(project_id, cursor):
    cursor.execute("SELECT SUM(Amount) FROM Incomes WHERE Project_ID = ?;", (project_id,))
    total_income = cursor.fetchone()[0] or 0

    cursor.execute("SELECT SUM(Amount) FROM Costs WHERE Project_ID = ?;", (project_id,))
    total_cost = cursor.fetchone()[0] or 0

    if total_income > total_cost:
        status = 'Прибыльный'
    else:
        status = 'Убыточный'
        
    cursor.execute("UPDATE Projects SET Status = ? WHERE ID = ?;", (status, project_id))

In [5]:
with sqlite3.connect(path, timeout=timeout) as conn:
    conn = sqlite3.connect(path)
    cursor = conn.cursor()

    cursor.execute("SELECT ID FROM Projects;")
    projects = cursor.fetchall()
    try:
        for project_id in projects:
            recalculate_project_status(project_id[0], cursor)
    
        conn.commit()
        display(pd.read_sql_query('SELECT * FROM Projects LIMIT 10;', conn) )
    except Exception as e:
        print(f"Ошибка при выполнении скрипта: {e}")    

,ID,Title,Description,Status
0,1,Мир Смеха,Проект о юморе и комедии.,Прибыльный
1,2,Горячие минуты,Проект о банях и отдыхе.,Убыточный
2,3,Темные дела,Проект о криминале и расследованиях.,Убыточный
3,4,Аргументы и факты,Проект о фактах и аргументах.,Прибыльный
4,5,Ночная история,Проект об историях на ночь.,Прибыльный
5,6,Женский голос,Проект о женском стендапе и разговорах.,Убыточный
6,7,В эфире,Проект о подкастах и обсуждениях.,Прибыльный
7,8,Легенды,Проект о биографиях великих людей.,Убыточный
8,9,Сцена и юмор,Проект о шоу и сценических выступлениях.,Прибыльный


Создание триггеров

In [6]:
with sqlite3.connect(path, timeout=timeout) as conn:
    cursor = conn.cursor()
    
    # Шаг 2: Создание триггеров
    trigger_incomes = """
    CREATE TRIGGER AfterInsertIncomes
    AFTER INSERT ON Incomes
    FOR EACH ROW
    BEGIN
        UPDATE Projects
        SET Status = CASE
            WHEN (SELECT SUM(Amount) FROM Incomes WHERE Project_ID = NEW.Project_ID) >
                 (SELECT SUM(Amount) FROM Costs WHERE Project_ID = NEW.Project_ID)
            THEN 'Прибыльный'
            ELSE 'Убыточный'
        END
        WHERE ID = NEW.Project_ID;
    END;
    """
    trigger_costs = """
    CREATE TRIGGER AfterInsertCosts
    AFTER INSERT ON Costs
    FOR EACH ROW
    BEGIN
        UPDATE Projects
        SET Status = CASE
            WHEN (SELECT SUM(Amount) FROM Incomes WHERE Project_ID = NEW.Project_ID) >
                 (SELECT SUM(Amount) FROM Costs WHERE Project_ID = NEW.Project_ID)
            THEN 'Прибыльный'
            ELSE 'Убыточный'
        END
        WHERE ID = NEW.Project_ID;
    END;
    """
    cursor.execute(trigger_incomes)
    cursor.execute(trigger_costs)

Проверяем, что они добавились

In [7]:
with sqlite3.connect(path, timeout=timeout) as conn:
    cursor = conn.cursor()
    
    cursor.execute("SELECT name, sql FROM sqlite_master WHERE type='trigger';")
    triggers = cursor.fetchall()

    # Вывод списка триггеров
    for trigger in triggers:
        print(f"Триггер: {trigger[0]}\nSQL: {trigger[1]}\n")

Триггер: AfterInsertIncomes
SQL: CREATE TRIGGER AfterInsertIncomes
    AFTER INSERT ON Incomes
    FOR EACH ROW
    BEGIN
        UPDATE Projects
        SET Status = CASE
            WHEN (SELECT SUM(Amount) FROM Incomes WHERE Project_ID = NEW.Project_ID) >
                 (SELECT SUM(Amount) FROM Costs WHERE Project_ID = NEW.Project_ID)
            THEN 'Прибыльный'
            ELSE 'Убыточный'
        END
        WHERE ID = NEW.Project_ID;
    END

Триггер: AfterInsertCosts
SQL: CREATE TRIGGER AfterInsertCosts
    AFTER INSERT ON Costs
    FOR EACH ROW
    BEGIN
        UPDATE Projects
        SET Status = CASE
            WHEN (SELECT SUM(Amount) FROM Incomes WHERE Project_ID = NEW.Project_ID) >
                 (SELECT SUM(Amount) FROM Costs WHERE Project_ID = NEW.Project_ID)
            THEN 'Прибыльный'
            ELSE 'Убыточный'
        END
        WHERE ID = NEW.Project_ID;
    END



Получим список проектов и их статус

In [8]:
with sqlite3.connect(path, timeout=timeout) as conn:
    cursor = conn.cursor()        
    display(pd.read_sql_query('SELECT ID,Title, Status FROM Projects WHERE ID=3;', conn) )

,ID,Title,Status
0,3,Темные дела,Убыточный


In [9]:
with sqlite3.connect(path, timeout=timeout) as conn:
    cursor = conn.cursor()        
    display(pd.read_sql_query('SELECT SUM(Amount) FROM Incomes WHERE Project_ID=3 ;', conn) )
    display(pd.read_sql_query('SELECT SUM(Amount) FROM Costs WHERE Project_ID=3 ;', conn) )


,SUM(Amount)
0,1113335


,SUM(Amount)
0,2590745


In [10]:
with sqlite3.connect(path, timeout=timeout) as conn:
    cursor = conn.cursor()        
    display(pd.read_sql_query('SELECT * FROM Incomes WHERE Project_ID=3 ;', conn) )

,ID,Project_ID,Category,Counterparty,CreationDate,Amount
0,2,3,Продажа цифрового контента,Андреев Лимитед,2019-11-09,768009
1,23,3,Реклама в социальных сетях,ИКЕА Дом (INGKA),2016-10-31,345326


In [11]:
cursor.execute("INSERT INTO Incomes (ID, Project_ID, Category, Counterparty, CreationDate, Amount) VALUES (?, ?, ?, ?, ?, ?);",
               (31, 3, 'Продажа цифрового контента', 'Андреев Лимитед', '2019-11-09', 1477410+50000))
conn.commit()

Триггер сработал и статус корректно обновился

In [12]:
with sqlite3.connect(path, timeout=timeout) as conn:
    cursor = conn.cursor()        
    display(pd.read_sql_query('SELECT ID, Title, Status FROM Projects WHERE ID=3;', conn) )

,ID,Title,Status
0,3,Темные дела,Прибыльный
